# GAIA PSF Simulation

First attempt to simulate close binary stars observation using the gaia telescope. The latter is imagined as a rectangular pupil, with no pixel binning or aberrations

#### Initialization

In [1]:
%matplotlib qt

QSocketNotifier: Can only be used with threads started with QThread


In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format=' - %(levelname)s - %(message)s')

In [111]:
from more_itertools import factor
import poppy
import xupy as xp
from xupy import typing as xt
import astropy.units as u
from astropy.table import QTable
from astropy.io import fits
from matplotlib import pyplot as plt
from astropy import convolution as c
from opticalib import load_fits
from poppy.utils import rebin_array

Gpb = QTable.read("data/gaiaDR3passband.fits")
weights = Gpb.filled(0)

def rebinned(psf: fits.HDUList | xt.ArrayLike, rebin_factor: int) -> fits.HDUList:
    if isinstance(psf, fits.HDUList):
        psf = psf[0].data
    px_ratio = (rebin_factor, 3*rebin_factor)
    return rebin_array(psf, px_ratio)

In [40]:
poppy.conf.use_multiprocessing = False
poppy.conf.n_processes = 20
poppy.conf.double_precision = False

M1 = poppy.RectangleAperture(name="Primary Mirror", width=1.45*u.m, height=0.5*u.m)
M2 = poppy.RectangleAperture(name="Beam Collimator / Secondary Mirror",width=0.35*u.m, height=0.2*u.m)

In [ ]:
telescope = poppy.OpticalSystem(name="Gaia", oversample=1, verbose=True)

telescope.add_pupil(M1)
telescope.add_detector(name="SkyPLANE", pixelscale=0.001*(u.arcsec/u.pixel), fov_arcsec=5*u.arcsec, oversample=2)

psf = telescope.calc_psf(progressbar=True, return_final=True, source={'wavelengths': Gpb['lambda'], 'weights': weights['G']})

In [ ]:
ccd_psf = rebinned(psf[0], rebin_factor=4)

In [117]:
plt.imshow(ccd_psf, origin='lower', cmap='inferno')
plt.colorbar()

In [34]:
ccd = poppy.OpticalSystem(name="CCD", oversample=4, verbose=True)
ccd.add_pupil(M1)
ccd.add_detector(name="CCD", pixelscale=0.059*(u.arcsec/u.pixel), fov_arcsec=6*u.arcsec, oversample=59)
ccd.add_rotation(90)

psf_ccd = ccd.calc_psf(progressbar=True, return_final=True, source={'wavelengths': Gpb['lambda'], 'weights': weights['G']})
ccd_psf = psf_ccd[0][0].data
ccd_psf = poppy.utils.rebin_array(ccd_psf, (1,3))

plt.figure(figsize=(8,8))
plt.imshow(ccd_psf, origin='lower', cmap='inferno')
plt.colorbar(shrink=0.8, label='Intensity')
plt.title("CCD PSF")
plt.xlabel("Pixels")
plt.ylabel("Pixels")
plt.show()

  0%|                                                   | 0/781 [00:00<?, ?it/s]

100%|█████████████████████████████████████████| 781/781 [01:07<00:00, 11.58it/s]
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [35]:
poppy.display_psf(psf_ccd[0], title='PSF on CCD')

# Prova